In [1]:
# loan data mining 
# part 5
# Grid Search CV

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [14]:
# use train data
train=pd.read_csv('loantrain.csv')
test=pd.read_csv('loantest.csv')

In [15]:
train=train.drop('custid',axis=1)
test=test.drop('custid',axis=1)

In [16]:
train = train.replace(np.nan, 0)
test = test.replace(np.nan, 0)

In [21]:
Xtrain = train.drop('status',axis=1)
ytrain = train['status']
Xtest = test.drop('status',axis=1)
ytest = test['status']

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

In [22]:
#Gradient Boosting Classifier
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }
modelgsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=5, 
                                     scoring="accuracy", n_jobs= -1, verbose = 1)
modelgsGBC.fit(Xtrain,ytrain)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.4s


KeyboardInterrupt: 

In [23]:
# Logistic Regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
scores = cross_val_score(logreg, Xtrain, ytrain, cv=5)
scores

array([0.77627628, 0.76276276, 0.77327327, 0.78947368, 0.78045113])

In [24]:
# Random Forests
from sklearn.ensemble import RandomForestClassifier
from sklearn import  metrics

# refer to: https://blog.csdn.net/qq_35040963/article/details/88832030

rf0 = RandomForestClassifier(oob_score=True, random_state=2019)
rf0.fit(Xtrain,ytrain.astype('int'))
print (rf0.oob_score_)

0.7418870192307693


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


In [25]:
y_predprob = rf0.predict_proba(Xtest)[:,1]
print( "AUC Score (Train): %f" % metrics.roc_auc_score(ytest, y_predprob))

AUC Score (Train): 0.727648


In [26]:
# GridSearch for n_estimators
param_test1 = {'n_estimators':[50,120,160,200,250]}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(Xtrain,ytrain)
print( gsearch1.best_params_, gsearch1.best_score_)

{'n_estimators': 160} 0.7805210975557626


In [27]:
param_test2 = {'max_depth':[1,2,3,5,7,9,11,13]}#, 'min_samples_split':[100,120,150,180,200,300]}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators=50, min_samples_split=100,
                                  min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=10),
   param_grid = param_test2, scoring='roc_auc',iid=False, cv=5)
gsearch2.fit(Xtrain,ytrain)
print( gsearch2.best_params_, gsearch2.best_score_)

{'max_depth': 7} 0.7788225827133612


In [28]:
rf1 = RandomForestClassifier(n_estimators= 160, max_depth=7, min_samples_split=100, min_samples_leaf=20,max_features='sqrt' ,oob_score=True, random_state=2019)
rf1.fit(Xtrain,ytrain)
print( rf1.oob_score_)
y_predprob = rf1.predict_proba(Xtest)[:,1]
print( "AUC Score (Train): %f" % metrics.roc_auc_score(ytest, y_predprob))

0.7863581730769231
AUC Score (Train): 0.799465


In [30]:
param_test3 = {'min_samples_split':[80,100,120], 'min_samples_leaf':[10,20,30,50]}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 160, max_depth=7,
                                  max_features='sqrt' ,oob_score=True, random_state=2017),
   param_grid = param_test3, scoring='roc_auc',iid=False, cv=5)
gsearch3.fit(Xtrain,ytrain)
print( gsearch3.best_params_, gsearch3.best_score_)

{'min_samples_leaf': 30, 'min_samples_split': 80} 0.7800080662916592


In [31]:
param_test4 = {'max_features':[3,5,7,9]}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 160, max_depth=7, min_samples_split=80,
                                  min_samples_leaf=30 ,oob_score=True, random_state=2019),
   param_grid = param_test4, scoring='roc_auc',iid=False, cv=5)
gsearch4.fit(Xtrain,ytrain)
print( gsearch4.best_params_, gsearch4.best_score_)

{'max_features': 7} 0.7810637585435192


In [32]:
# the optimal RF
rf2 = RandomForestClassifier(n_estimators= 160, max_depth=7, min_samples_split=80,
                                min_samples_leaf=30,max_features=7,oob_score=True, random_state=2019)
rf2.fit(Xtrain,ytrain)
print (rf2.oob_score_)

0.78515625


In [33]:
# SVM
from sklearn.metrics import classification_report
from sklearn.svm import SVC
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']

In [ ]:
clf = GridSearchCV(SVC(), tuned_parameters, cv=5
                       )
clf.fit(Xtrain, ytrain)

print("Best parameters set found on development set:")
print(clf.best_params_)

In [ ]:
y_pred=clf.predict(X_test)
print(classification_report(y_test, y_pred))